In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchvision.models import resnet18
from torchcam.cams import SmoothGradCAMpp
from torchcam.cams import ScoreCAM
from torchcam.cams import CAM
from torchcam.utils import overlay_mask

from pfe.EarlyStop.EarlyStop import EarlyStopping
!pip install imutils
!pip install pytorchtools
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision
import torch
from imutils import paths
import random
import cv2
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader
from pfe.Dataset.Dataset import XrayData
from tqdm import tqdm
import torch.nn.functional as F
from math import floor, ceil
import shutil
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import os
from pfe.utils.utils import get_mean_std

  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
    Preparing wheel metadata ... - done
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
     |████████████████████████████████| 75 kB 1.1 MB/s eta 0:00:01
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
     |████████████████████████████████| 145 kB 1.3 MB/s eta 0:00:01
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
     |████████████████████████████████| 138 kB 985 kB/s eta 0:00:01
  Created wheel for gdown: filename=gdown-3.13.0-py3-none-any.whl size=9034 sha256=a7ace3eaf025d30874d0af8241cd1687fb06907928950bd5b44f017935197361
  Stored in directory: /Users/benx13/Library/Caches/pip/wheels/04/51/53/ed3e97af28b242e9eb81afb4836273fbe233a14228aa82fea3
Successfully built gdown
You should consider upgrading via the '/Users/benx13/PycharmProjects/pythonProject/venv/bin/python -m pip install --u

In [1]:
BATCH = 32
EPOCH = 64
lr = 1e-4

In [2]:
t = Compose([ transforms.ToPILImage(),transforms.Resize((224,224),interpolation=Image.NEAREST), transforms.ToTensor()])

dataset = XrayData(['/kaggle/working/CovRad/BS', '/kaggle/working/Mendeley/BS'], transforms=t, enhanced=False, segmented=False, gamma=2)
train_set, test_set = torch.utils.data.random_split(dataset, [floor(len(dataset) * 0.8), ceil(len(dataset)*0.2)])
train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=True)
mean, std = get_mean_std(train_loader)
mean, std

0.0001

In [3]:
batch, _ = next(iter(dataset))
plt.imshow(batch[0], cmap='gray')


NameError: name 'dataset' is not defined

In [ ]:
t = Compose([transforms.ToPILImage(),
                   transforms.Resize((224,224),
                   interpolation=Image.NEAREST),
                   transforms.RandomHorizontalFlip(),
                   transforms.RandomVerticalFlip(),
                   transforms.ToTensor(),
                   transforms.Normalize(mean, std)])

dataset = XrayData(['/kaggle/working/CovRad/BS', '/kaggle/working/Mendeley/BS'], transforms=t, enhanced=False, segmented=False, gamma=2)
train_set, test_set = torch.utils.data.random_split(dataset, [floor(len(dataset) * 0.8), ceil(len(dataset)*0.2)])
train_loader = DataLoader(train_set, batch_size=BATCH, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH, shuffle=True)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

NameError: name 'torch' is not defined

In [6]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 4)
model.to(device)
criterion = nn.CrossEntropyLoss()
SGDM = torch.optim.Adam(model.parameters(), lr=lr)

NameError: name 'models' is not defined

In [9]:
def train(model=None, loss=None, optimizer=None, epochs=64, patience=5):
    # Train the model
    total_step = len(train_loader)
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    conf_mats = []
    checkpoint_path = './best_checkpoint_dict.pt'
    best_model_path = './best_model.pt'
    val_loss_min = 1

    early_stopping = EarlyStopping(patience=patience, verbose=True)

    for epoch in tqdm(range(epochs)):
        acc = []
        correct = 0
        total = 0
        # Training step
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)


            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, outputs = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (labels == outputs).sum()
            acc.append((correct/total).item())


        print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.4f}".format(epoch+1, EPOCH, i+1, total_step, loss.item(), np.mean(acc)))
        train_loss.append(loss.item())
        train_acc.append(np.mean(acc))

        # Validation step
        correct = 0
        total = 0
        v_loss = []
        v_acc = []
        confusion_matrix = torch.zeros(4, 4)
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            predictions = model(images)
            _, predicted = torch.max(predictions, 1)
            total += labels.size(0)
            correct += (labels == predicted).sum()
            loss = criterion(predictions, labels)
            v_loss.append(loss.item())
            v_acc.append((correct/total).item())

            for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
            conf_mats.append(confusion_matrix)


        print('Val Loss: {:.4f}, Val Acc: {:.4f}'.format(np.mean(v_loss), correct/total))
        val_acc.append((correct/total).item())
        val_loss.append(np.mean(v_loss))


        train_history = {
        'train_loss': train_loss,
        'val_loss': val_loss,
        'train_acc': train_acc,
        'val_acc': val_acc,
        'confusion_matrix':conf_mats
        }
        checkpoint = {
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'Trainig_History' : train_history,
        }

        # save checkpoint
        #save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        ## TODO: save the model if validation loss has decreased
        #if np.mean(v_loss) < val_loss_min:
            #print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(val_loss_min,np.mean(v_loss)))
            # save checkpoint as best model
            #save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            #val_loss_min = np.mean(v_loss)


        early_stopping(np.mean(v_loss), model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

    model.load_state_dict(torch.load('checkpoint.pt'))
    torch.save(train_history, 'train_h.pt')
    return model, train_history

In [10]:
model, t_h = train(model=model, loss=criterion, optimizer=SGDM, epochs=128, patience=20)

NameError: name 'model' is not defined

In [12]:
# visualize the loss as the network trained
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(t_h['train_loss'])+1),t_h['train_loss'], label='Training Loss')
plt.plot(range(1,len(t_h['val_loss'])+1),t_h['val_loss'],label='Validation Loss')
plt.plot(range(1,len(t_h['train_acc'])+1),t_h['train_acc'], label='Training Accuracy')
plt.plot(range(1,len(t_h['val_acc'])+1),t_h['val_acc'],label='Validation Accuracy')

# find position of lowest validation loss
minposs = t_h['val_loss'].index(min(t_h['val_loss']))+1
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss/acc')
plt.ylim(0, 1.1) # consistent scale
plt.xlim(0, len(t_h['train_loss'])+1) # consistent scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('loss_plot.png', bbox_inches='tight')

NameError: name 'plt' is not defined

In [13]:
print('Min Loss : ', np.min(t_h['val_loss']), ' Avg : ', np.mean(t_h['val_loss'][3:15]))
print('Max Acc : ', np.max(t_h['val_acc']), ' Avg : ', np.mean(t_h['val_acc'][3:15]))

NameError: name 'np' is not defined

In [14]:
flatten = lambda t: [item for sublist in t for item in sublist]
y_pred_list = []
y_test_list = []
with torch.no_grad():
    model.eval()
    for X_batch, y_tag in test_loader:
        #print(y_tag.tolist())
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch).cpu()
        _, y_pred_tags = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tags.cpu().numpy())
        y_test_list.append(y_tag.tolist())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
y_pred_list = flatten(y_pred_list)
y_test_list = flatten(y_test_list)

NameError: name 'torch' is not defined

In [15]:
target_names={0:'Normal', 1:'Covid', 2:"Lung_Opacity", 3:'Pneumonia'}
confusion_matrix_df = pd.DataFrame(
    confusion_matrix(np.array(y_test_list),
                     np.array(y_pred_list))).rename(
    columns=target_names, index=target_names)

plt.figure(figsize=(10,8))
sns.heatmap(confusion_matrix_df, annot=True, fmt='g')


NameError: name 'pd' is not defined

In [16]:
print(classification_report(y_test_list, y_pred_list, target_names=['Normal', 'Covid', "Lung_Opacity", 'Pneumonia'], digits=4))


NameError: name 'classification_report' is not defined

In [17]:
cam_extractor = ScoreCAM(model)
X_batch, y_tag = next(iter(test_loader))

NameError: name 'ScoreCAM' is not defined

In [18]:

for i in range(32):
    img = X_batch[i]
    y_test_pred = model(img.unsqueeze(0).to(device)).cpu()
    _, y_pred_tags = torch.max(y_test_pred, dim = 1)
    if(y_pred_tags.item()==3):
        print(target_names[y_pred_tags.item()])
        activation_map = cam_extractor(y_test_pred.squeeze(0).argmax().item(), y_test_pred)
        z = img * torch.tensor(std).view(3, 1, 1)
        z = z + torch.tensor(mean).view(3, 1, 1)
        result = overlay_mask(to_pil_image(z), to_pil_image(activation_map, mode='F'), alpha=0.5)
        # Display it
        plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()

NameError: name 'X_batch' is not defined